In [16]:
#PACKAGES

import pandas as pd
import yfinance as yf
import talib
import numpy as np
import requests
from io import StringIO
from datetime import date, datetime, timedelta
import praw
import emoji
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob
import matplotlib.pyplot as plt

In [17]:
#READ

fundamental = pd.read_csv("/Users/pedroalexleite/Desktop/Portfolio-Construction-1/Data/fundamental.csv")
technical = pd.read_csv("/Users/pedroalexleite/Desktop/Portfolio-Construction-1/Data/technical.csv")
sentimental = pd.read_csv("/Users/pedroalexleite/Desktop/Portfolio-Construction-1/Data/sentimental.csv")

df = (
    fundamental
    .merge(technical, on="Symbol", how="outer")
    .merge(sentimental, on="Symbol", how="outer")
)

df = df.dropna()

In [18]:
#PERCENTIL

def percentil(series, invert=False):
    ranks = series.rank(pct=True) 
    if invert:
        ranks = 1 - ranks
    scores = np.floor(ranks * 100).astype(int) + 1  
    scores = scores.clip(1, 100)
    
    return scores

cols_invert = ["PER", "PBR", "EV/EBITDA", "P/S"]
cols_normal = [
    "PEGR", "DY", "ROA", "NPM", "ICR", "FCF", "RGR", "EGR",
    "SMA", "EMA", "MACD", "ADX", "RSI", "ROC", "OBV", "MFI",
    "Sentimental"
]

for col in cols_invert:
    df[col] = percentil(df[col], invert=True)

for col in cols_normal:
    df[col] = percentil(df[col], invert=False)

In [19]:
#RATING

fundamental_score = (
    df[["PER", "PEGR", "PBR", "DY", "EV/EBITDA", "P/S"]].mean(axis=1) * 0.4 +
    df[["ROA", "NPM"]].mean(axis=1) * 0.2 +
    df[["ICR", "FCF"]].mean(axis=1) * 0.2 +
    df[["RGR", "EGR"]].mean(axis=1) * 0.2
)

technical_score = (
    df[["SMA", "EMA", "MACD", "ADX"]].mean(axis=1) * 0.5 +
    df[["RSI", "ROC"]].mean(axis=1) * 0.3 +
    df[["OBV", "MFI"]].mean(axis=1) * 0.2
)

sentimental_score = df["Sentimental"]

df2 = pd.DataFrame({
    "Symbol": df["Symbol"],
    "Fundamental": fundamental_score,
    "Technical": technical_score,
    "Sentimental": sentimental_score
})

df2["Fundamental"] = percentil(df2["Fundamental"])
df2["Technical"] = percentil(df2["Technical"])

df2["Rating"] = (
    df2["Fundamental"] * 0.5 +
    df2["Technical"] * 0.3 +
    df2["Sentimental"] * 0.2
)

df2["Rating"] = percentil(df2["Rating"])

In [20]:
#SAVE

#df.to_csv("/Users/pedroalexleite/Desktop/Portfolio-Construction-1/Data/aux_rating.csv", index=False)
df2.to_csv("/Users/pedroalexleite/Desktop/Portfolio-Construction-1/Data/rating.csv", index=False)